# Face Swap paso a paso
La finalidad de este código es ponerle a la persona a la que está grabando la webcam, la cara de algún personaje famoso que obtendremos de una foto. No obstante, para mejor comprensión, se ha empezado por hacer un face swap entre dos imágenes. Los pasos que se han seguido para conseguir nuestro objetivo se describen a continuación.

# Parte 1: Landmarks y ConvexHull
El objetivo inicial es encontrar la cara en la imagen 1 (Harry Potter en este caso) y extraerla en una máscara que será modificada para ponerse en otra persona.



In [ ]:
import cv2
import dlib
import numpy as np

# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara 1. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        
        # Para rodear los landmarks en la imagen, con centro en (x,y), 3 pixels, color rojo y grosor -1 (relleno)
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1) 
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Para dibujar el poligono convex hull. El true indica que cerremos el poligono, luego color y grosor
    cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)





cv2.imshow("Imagen origen", img)
cv2.imshow("Mascara", mask)
cv2.imshow("Mascara + Cara", face_image_1)


# Muestramelo hasta que pulse alguna tecla
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="../fotos/parte1.jpg" width="200"/>

# Parte 2 - Triangulacion Delauny, imagen origen
El siguiente paso es la triangulación de la cara origen después de extraer su convexHull en una máscara. La triangulación Delauny es un importante algoritmo de geometría computacional. Consiste en una red de triángulos conexa y convexa sujeta a la condición de que la circunferencia circunscrita de cada triángulo no debe contener ningún otro vértice de la triangulación en su interior, aunque sí puede haber vértices situados en la circunferencia.

El objetivo de la triangulación de la cara origen es poder hacer coincidir los triángulos de la cara origen con la cara destino. Al hacer que los triángulos en ambas caras encajen, se guardarán las proporciones entre ambas, haciendo posible que se ajusten diferentes expresiones faciales de la cara 1 a la cara 2.

In [5]:
import cv2
import dlib
import numpy as np

# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara de Harry. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        
        # Para rodear los landmarks en la imagen, con centro en (x,y), 3 pixels, color rojo y grosor -1 (relleno)
        #cv2.circle(img, (x, y), 5, (0, 255, 0), -1) 
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Para dibujar el poligono convex hull. El true indica que cerremos el poligono, luego color y grosor
    #cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        # Lineas que unen los vertices
        cv2.line(img, pt1, pt2, (0, 0, 255), 2)
        cv2.line(img, pt2, pt3, (0, 0, 255), 2)
        cv2.line(img, pt1, pt3, (0, 0, 255), 2)

cv2.imshow("Imagen 1", img)
cv2.imshow("Mascara", mask)
cv2.imshow("Mascara + Cara", face_image_1)


# Muestramelo hasta que pulse alguna tecla
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="../fotos/parte2.jpg" width="200"/>

# Parte 3: Triangulacion imagen destino a partir de la imagen origen
Ya hemos dividido la primera cara en triangulos. Ahora queremos extraer los mismos triangullillos en la segunda imagen. El problema es que no se puede aplicar tal cual el mismo algoritmo Delauny en la segunda imagen. Esto es porque si lo aplicásemos tal ual en la segunda imagen los triangulos serian otros distintos (otras distancias) y las caras no guardarian las mismas proporciones.

**¿Solution?:** 
1. Encontramos los landmark points que se corresponden con cada vértice de los triangulos en la primera imagen.

In [7]:
import cv2
import dlib
import numpy as np


# Funcion que extrae indices de un np array
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
img2 = cv2.imread("../faces/hermione.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara de Harry. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        
        # Para rodear los landmarks en la imagen, con centro en (x,y), 3 pixels, color rojo y grosor -1 (relleno)
        #cv2.circle(img, (x, y), 3, (0, 0, 255), -1) 
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Para dibujar el poligono convex hull. El true indica que cerremos el poligono, luego color y grosor
    #cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        
        # Encontramos los indices de los vértices
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        
        #Si todo ok
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3] # Estos son los landmarks de los triangulos
            indexes_triangles.append(triangle)           # Los guardamos
       
        
        # Lineas que unen los vertices
        cv2.line(img, pt1, pt2, (0, 0, 255), 2)
        cv2.line(img, pt2, pt3, (0, 0, 255), 2)
        cv2.line(img, pt1, pt3, (0, 0, 255), 2)
        
# Vamos a buscar los triangulos de la segunda cara usando los landmarks
faces2 = detector(img2_gray) # Detectamos cara de la segunda imagen
for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        #cv2.circle(img2, (x, y), 3, (0, 255, 0), -1)

# Triangularizacion de la segunda cara (sin usar Delauny porque no sirve ya) a partir de la primera
for triangle_index in indexes_triangles:
    # Hacemos coincidir cada punto de los triangulos con las mismas posiciones que en la cara de la primera imgagen
    pt1 = landmarks_points[triangle_index[0]] 
    pt2 = landmarks_points[triangle_index[1]]
    pt3 = landmarks_points[triangle_index[2]]
    cv2.line(img2, pt1, pt2, (0, 0, 255), 2)
    cv2.line(img2, pt3, pt2, (0, 0, 255), 2)
    cv2.line(img2, pt1, pt3, (0, 0, 255), 2)



cv2.imshow("Imagen 1", img)
#cv2.imshow("Mascara", mask)
#cv2.imshow("Mascara + Cara", face_image_1)
cv2.imshow("Imagen 2", img2)


# Muestramelo hasta que pulse alguna tecla
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="../fotos/parte3.jpg" width="300"/>

# Parte 4: Hacer que los triangulos se correspondan + deformacion
Una vez trianguladas ambas caras de la misma forma, se procede a hacer corresponder la triangulación origen sobre la triangulación destino. Aunque los triángulos se trazan a partir de los mismos landmark points, no tienen la misma forma, así que se hará una deformación (warp) para hacerlos coincidir.


In [9]:
import cv2
import dlib
import numpy as np


# Funcion que extrae indices de un np array
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
img2 = cv2.imread("../faces/hermione.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara de Harry. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        
        # Para rodear los landmarks en la imagen, con centro en (x,y), 3 pixels, color rojo y grosor -1 (relleno)
        #cv2.circle(img, (x, y), 3, (0, 0, 255), -1) 
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Para dibujar el poligono convex hull. El true indica que cerremos el poligono, luego color y grosor
    #cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        
        # Encontramos los indices de los vértices
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        
        # Si todo ok       
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3] # Estos son los landmarks de los triangulos
            indexes_triangles.append(triangle)           # Los guardamos
       
        
# Vamos a buscar los triangulos de la segunda cara usando los landmarks
faces2 = detector(img2_gray) # Detectamos cara de la segunda imagen
for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))
        #cv2.circle(img2, (x, y), 3, (0, 255, 0), -1)


# -------------------------------------------TRIANGUARIZACIÓN DE AMBAS CARAAS EN UN UNICO BUCLE-------------------------------
for triangle_index in indexes_triangles:
    
    # PRIMERA CARA, triangulo 1
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    
    # Triangulo con sus 3 vertices
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
    # Este es el rectangulo que rodea el triangulo
    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1 # x e y del rectangulo, anchura y altura
    cropped_triangle = img[y: y + h, x: x + w]    # Este es el triangulo recortado del rectangulo que acabamos de crear
    cropped_tr1_mask = np.zeros((h, w), np.uint8) # Esta es una mascara de fondo negro del tamaño del triangulo.
    
    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr1_mask, points, 255) # Relleno la mascara con los puntos
    # Como antes, esto sirve para rellenar el hueco blanco de la mascara negra con la imagen en color con foma de triangulo
    cropped_triangle = cv2.bitwise_and(cropped_triangle, cropped_triangle,mask=cropped_tr1_mask) 
    
    # Para dibujar las lineas de los triangulos
    cv2.line(img, tr1_pt1, tr1_pt2, (0, 0, 255), 2)
    cv2.line(img, tr1_pt3, tr1_pt2, (0, 0, 255), 2)
    cv2.line(img, tr1_pt1, tr1_pt3, (0, 0, 255), 2)
 
    # TRIANGULATION SEGUNDA CARA, triangulo2. Mismo proceso
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
    
    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2
    cropped_triangle2 = img2[y: y + h, x: x + w]
    cropped_tr2_mask = np.zeros((h, w), np.uint8)
    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                        [tr2_pt2[0] - x, tr2_pt2[1] - y],
                        [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)
    cropped_triangle2 = cv2.bitwise_and(cropped_triangle2, cropped_triangle2, mask=cropped_tr2_mask)

    cv2.line(img2, tr2_pt1, tr2_pt2, (0, 0, 255), 2)
    cv2.line(img2, tr2_pt3, tr2_pt2, (0, 0, 255), 2)
    cv2.line(img2, tr2_pt1, tr2_pt3, (0, 0, 255), 2)
    
    # WARP: DEFORMACION DE LOS TRIANGULOS
    # Conversión a float 32
    points = np.float32(points)
    points2 = np.float32(points2)
    # Matrix de transformación (deformación)
    M = cv2.getAffineTransform(points, points2)
    # Cogemos el triangulo de la imagen 1 (cropped_triangle) y lo transformamos a la width and height del segundo
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    break


cv2.imshow("Imagen 1", img)
#cv2.imshow("Mascara", mask)
#cv2.imshow("Mascara + Cara", face_image_1)
cv2.imshow("Recorte 1", cropped_triangle)
cv2.imshow("Recorte 2", cropped_triangle2)
cv2.imshow("Warp - Deformacion de 1 a 2", warped_triangle)
cv2.imshow("Imagen 2", img2)


# Muestramelo hasta que pulse alguna tecla
cv2.waitKey(0)
cv2.destroyAllWindows()

# Part 5: Juntamos los triangulos deformados
El objetivo ahora es conseguir deformar cada uno de los triangulos de la primera cara y hacerlos coincidir con la segunda cara.

Las partes negras de las mascaras tienen un problema, cubren la imagen y cortan ese trozo. 

In [12]:
import cv2
import dlib
import numpy as np
import time


# Funcion que extrae indices de un np array
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
img2 = cv2.imread("../faces/hermione.jpg")

# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara 1. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

#Mascara negra con el mismo tamaño y color de la segunda imagen
img2_new_face = np.zeros_like(img2)

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        
        # Encontramos los indices de los vértices
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        
        # Si todo ok       
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3] # Estos son los landmarks de los triangulos
            indexes_triangles.append(triangle)           # Los guardamos
       
        
# Vamos a buscar los triangulos de la segunda cara usando los landmarks
faces2 = detector(img2_gray) # Detectamos cara de la segunda imagen
for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))
        
    points2 = np.array(landmarks_points2, np.int32)
    convexhull2 = cv2.convexHull(points2)
        


# -------------------------------------------TRIANGUARIZACIÓN DE AMBAS CARAAS EN UN UNICO BUCLE-------------------------------
for triangle_index in indexes_triangles:
    
    # PRIMERA CARA, triangulo 1
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    
    # Triangulo con sus 3 vertices
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
    # Este es el rectangulo que rodea el triangulo
    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1 # x e y del rectangulo, anchura y altura
    cropped_triangle = img[y: y + h, x: x + w]    # Este es el triangulo recortado del rectangulo que acabamos de crear
    cropped_tr1_mask = np.zeros((h, w), np.uint8) # Esta es una mascara de fondo negro del tamaño del triangulo.
    
    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr1_mask, points, 255) # Relleno la mascara con los puntos
    # Como antes, esto sirve para rellenar el hueco blanco de la mascara negra con la imagen en color con foma de triangulo
    cropped_triangle = cv2.bitwise_and(cropped_triangle, cropped_triangle,mask=cropped_tr1_mask) 
    
 
    # TRIANGULATION SEGUNDA CARA, triangulo2. Mismo proceso
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
    
    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2
    cropped_triangle2 = img2[y: y + h, x: x + w]
    cropped_tr2_mask = np.zeros((h, w), np.uint8)
    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                        [tr2_pt2[0] - x, tr2_pt2[1] - y],
                        [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)
    cropped_triangle2 = cv2.bitwise_and(cropped_triangle2, cropped_triangle2, mask=cropped_tr2_mask)

    
    # WARP: DEFORMACION DE LOS TRIANGULOS
    # Conversión a float 32
    points = np.float32(points)
    points2 = np.float32(points2)
    # Matrix de transformación (deformación)
    M = cv2.getAffineTransform(points, points2)
    # Cogemos el triangulo de la imagen 1 (cropped_triangle) y lo transformamos a la width and height del segundo
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    
    
    # RECONSTRUCCION DE LA CARA DESTINO CON LOS TRIANGULOS DEFORMADITOS
    img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
    img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
    img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area # En la posicion especificada de la cara 2, plantamos el triangulo
    
# Face swapped (putting 1st face into 2nd face)
img2_face_mask = np.zeros_like(img2_gray)
img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
img2_face_mask = cv2.bitwise_not(img2_head_mask)

img2_head_noface = np.zeros_like(img)

img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
result = cv2.add(img2_head_noface, img2_new_face)

cv2.imshow("Imagen 1", img)
cv2.imshow("Imagen 2", img2)
cv2.imshow("Resultado", result)



cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="../fotos/parte5.jpg" width="400"/>

# Parte 6: Reemplazamos la cara y suavizamos los bordes con seamlessclone

In [2]:
import cv2
import dlib
import numpy as np
#import time


# Funcion que extrae indices de un np array
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


# Leemos imagen
img = cv2.imread("../faces/harry.jpg")
img2 = cv2.imread("../faces/hermione.jpg")

# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Máscara donde vamos a extraer la cara 1. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

#Mascara negra con el mismo tamaño y color de la segunda imagen
#img2_new_face = np.zeros_like(img2)
height, width, channels = img2.shape
img2_new_face = np.zeros((height, width, channels), np.uint8)

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        
        # Encontramos los indices de los vértices
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        
        # Si todo ok       
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3] # Estos son los landmarks de los triangulos
            indexes_triangles.append(triangle)           # Los guardamos
       
        
# Vamos a buscar los triangulos de la segunda cara usando los landmarks
faces2 = detector(img2_gray) # Detectamos cara de la segunda imagen
for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))
        
    points2 = np.array(landmarks_points2, np.int32)
    convexhull2 = cv2.convexHull(points2)
        

lines_space_mask = np.zeros_like(img_gray)
lines_space_new_face = np.zeros_like(img2)
# -------------------------------------------TRIANGUARIZACIÓN DE AMBAS CARAAS EN UN UNICO BUCLE-------------------------------
for triangle_index in indexes_triangles:
    
    # PRIMERA CARA, triangulo 1
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    
    # Triangulo con sus 3 vertices
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
    # Este es el rectangulo que rodea el triangulo
    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1 # x e y del rectangulo, anchura y altura
    cropped_triangle = img[y: y + h, x: x + w]    # Este es el triangulo recortado del rectangulo que acabamos de crear
    cropped_tr1_mask = np.zeros((h, w), np.uint8) # Esta es una mascara de fondo negro del tamaño del triangulo.
    
    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr1_mask, points, 255) # Relleno la mascara con los puntos
    # Como antes, esto sirve para rellenar el hueco blanco de la mascara negra con la imagen en color con foma de triangulo
    #cropped_triangle = cv2.bitwise_and(cropped_triangle, cropped_triangle,mask=cropped_tr1_mask) 
    
    # Esto quita las lineas, probarlo
    cv2.line(lines_space_mask, tr1_pt1, tr1_pt2, 255)
    cv2.line(lines_space_mask, tr1_pt2, tr1_pt3, 255)
    cv2.line(lines_space_mask, tr1_pt1, tr1_pt3, 255)
    lines_space = cv2.bitwise_and(img, img, mask=lines_space_mask)
 
    # TRIANGULATION SEGUNDA CARA, triangulo2. Mismo proceso
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
    
    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2
    #cropped_triangle2 = img2[y: y + h, x: x + w]
    cropped_tr2_mask = np.zeros((h, w), np.uint8)
    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                        [tr2_pt2[0] - x, tr2_pt2[1] - y],
                        [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)
    
    # WARP: DEFORMACION DE LOS TRIANGULOS
    # Conversión a float 32
    points = np.float32(points)
    points2 = np.float32(points2)
    # Matrix de transformación (deformación)
    M = cv2.getAffineTransform(points, points2)
    # Cogemos el triangulo de la imagen 1 (cropped_triangle) y lo transformamos a la width and height del segundo
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

    
    
    # RECONSTRUCCION DE LA CARA DESTINO CON LOS TRIANGULOS DEFORMADITOS
    img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
    img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
    _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

    img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
    img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area # En la posicion especificada de la cara 2, plantamos el triangulo
    
# Face swapped (putting 1st face into 2nd face)
img2_face_mask = np.zeros_like(img2_gray)
img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
img2_face_mask = cv2.bitwise_not(img2_head_mask)

#img2_head_noface = np.zeros_like(img)

img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
result = cv2.add(img2_head_noface, img2_new_face)

(x, y, w, h) = cv2.boundingRect(convexhull2)
center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.NORMAL_CLONE)


cv2.imshow("Imagen 1", img)
cv2.imshow("Imagen 2", img2)
cv2.imshow("seamlessclone", seamlessclone)
cv2.imshow("result", result)


cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="../fotos/harrmione.jpg" width="400"/>

# Final: Webcam

In [3]:
import cv2
import dlib
import numpy as np

# Funcion que extrae indices de un np array
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


# Leemos imagen
img = cv2.imread("../faces/hillary_clinton.jpg")
# Imagen en grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Máscara donde vamos a extraer la cara 1. Es una imagen fondo negro del mismo tamaño de la imagen original.
mask = np.zeros_like(img_gray)  # Máscara de un solo canal

# Captura de video
cap = cv2.VideoCapture(0)

# Face detector de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Faces que encontramos con detector en la imagen en grises
faces = detector(img_gray)  # Es una lista o lista de listas si hay más de una cara

# Recorremos todas las caras que pueda haber en la imagen
for face in faces: 
    landmarks = predictor(img_gray, face)  # landmarks de cada cara en escala de grises
    landmarks_points = []                  # Lista donde pondremos cada punto de landmark   
    for n in range(0, 68):                 # Cada landmark tiene 68 puntos
        # Cogemos la longitud en x del landmark, y luego la de y.
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
    
    # OpenCv necesitan los puntos como numpy array. Numpy va más rápido.
    points = np.array(landmarks_points, np.int32)
    
    # convexHull = parte externa de los landmarks = parte que queremos recortar
    # No permite angulos de mas de 180 grados, por eso no hace el triangulo ceja-nariz-ceja
    convexhull = cv2.convexHull(points)
    
    # Aqui ponemos en nuestra máscara (el fondo negro) el convexHull
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # Esto pone la cara de Harry en el hueco blanco
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull) # Esto encierra el convexHull en un rectangulo porque eso se traga Delauny
    subdiv = cv2.Subdiv2D(rect)         # Esto hace la triangulizacion en el el rectangulo que rodea a convexhull
    subdiv.insert(landmarks_points)     # Insertamos los puntos que quereos triangularizar
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32) # Array de triangulicos
    
    # Recorremos cada triangulo
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1]) # Coordenadas del vertice 1
        pt2 = (t[2], t[3]) # Coordenadas del vertice 2
        pt3 = (t[4], t[5]) # Coordenadas del vertice 3
        
        # Encontramos los indices de los vértices
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        
        # Si todo ok       
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3] # Estos son los landmarks de los triangulos
            indexes_triangles.append(triangle)           # Los guardamos
       
        
# Vamos a buscar los triangulos de la segunda cara usando los landmarks
while True:
    _, img2 = cap.read()
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img2_new_face = np.zeros_like(img2)
    faces2 = detector(img2_gray) # Detectamos cara de la segunda imagen
    for face in faces2:
        landmarks = predictor(img2_gray, face)
        landmarks_points2 = []
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_points2.append((x, y))
            
        points2 = np.array(landmarks_points2, np.int32)
        convexhull2 = cv2.convexHull(points2)
        

    lines_space_mask = np.zeros_like(img_gray)
    lines_space_new_face = np.zeros_like(img2)
    # -------------------------------------------TRIANGUARIZACIÓN DE AMBAS CARAAS EN UN UNICO BUCLE-------------------------------
    for triangle_index in indexes_triangles:
        
        # PRIMERA CARA, triangulo 1
        tr1_pt1 = landmarks_points[triangle_index[0]]
        tr1_pt2 = landmarks_points[triangle_index[1]]
        tr1_pt3 = landmarks_points[triangle_index[2]]
        
        # Triangulo con sus 3 vertices
        triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
        # Este es el rectangulo que rodea el triangulo
        rect1 = cv2.boundingRect(triangle1)
        (x, y, w, h) = rect1 # x e y del rectangulo, anchura y altura
        cropped_triangle = img[y: y + h, x: x + w]    # Este es el triangulo recortado del rectangulo que acabamos de crear
        cropped_tr1_mask = np.zeros((h, w), np.uint8) # Esta es una mascara de fondo negro del tamaño del triangulo.
        
        points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                        [tr1_pt2[0] - x, tr1_pt2[1] - y],
                        [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)
        cv2.fillConvexPoly(cropped_tr1_mask, points, 255) # Relleno la mascara con los puntos
    
    
        # TRIANGULATION SEGUNDA CARA, triangulo2. Mismo proceso
        tr2_pt1 = landmarks_points2[triangle_index[0]]
        tr2_pt2 = landmarks_points2[triangle_index[1]]
        tr2_pt3 = landmarks_points2[triangle_index[2]]
        
        triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
        
        rect2 = cv2.boundingRect(triangle2)
        (x, y, w, h) = rect2
        #cropped_triangle2 = img2[y: y + h, x: x + w]
        cropped_tr2_mask = np.zeros((h, w), np.uint8)
        points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                            [tr2_pt2[0] - x, tr2_pt2[1] - y],
                            [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)
        cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)
    
        # WARP: DEFORMACION DE LOS TRIANGULOS
        # Conversión a float 32
        points = np.float32(points)
        points2 = np.float32(points2)
        # Matrix de transformación (deformación)
        M = cv2.getAffineTransform(points, points2)
        # Cogemos el triangulo de la imagen 1 (cropped_triangle) y lo transformamos a la width and height del segundo
        warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

    
    
    # RECONSTRUCCION DE LA CARA DESTINO CON LOS TRIANGULOS DEFORMADITOS
        img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
        img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
        _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

        img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
        img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area # En la posicion especificada de la cara 2, plantamos el triangulo
        
    # Face swapped (putting 1st face into 2nd face)
    img2_face_mask = np.zeros_like(img2_gray)
    img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
    img2_face_mask = cv2.bitwise_not(img2_head_mask)



    img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
    result = cv2.add(img2_head_noface, img2_new_face)

    (x, y, w, h) = cv2.boundingRect(convexhull2)
    center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

    #seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.NORMAL_CLONE)
    seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.MIXED_CLONE)

    cv2.imshow("img2", img2)
    cv2.imshow("clone", seamlessclone)
    cv2.imshow("result", result)



    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

<img src="../fotos/cris_hil.jpg" width="400"/>